In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

pd.set_option("mode.chained_assignment", None)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")

# Uciml repo
from ucimlrepo import fetch_ucirepo

# Pytorch device specific configuration
# Pytorch Gpu Configuration for cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pytorch Gpu Configuration for dml devices
# import torch_directml

# device = torch_directml.device()

# Set default devices
torch.set_default_device(device)

In [54]:
# Import heart desease data
heart_disease = fetch_ucirepo(id=45)

# Data as pandas dataframe
data = heart_disease.data.features
labels = heart_disease.data.targets

In [ ]:
#### Data pre-processing ####
# Filling nan values with it's mean
keys = data.keys()
for key in keys:
    # print(f"{key} before filling na count: {data[key].isna().sum()}")
    data[key] = data[key].fillna(data[key].mean())
    # print(f"{key} after filling na count: {data[key].isna().sum()}")

# Binarized labels
labels = labels.values > 0
data = data.values
print(data.shape, labels.shape)

(303, 13) (303, 1)


In [60]:
# convert data into tensors
dataT = torch.tensor(data).float()
labelsT = torch.tensor(labels).long()

# Train test split
train_data, test_data, train_labels, test_labels = train_test_split(dataT, labelsT, train_size=.9)

# Pytorch datasets
train_dataT = TensorDataset(train_data, train_labels)
test_dataT = TensorDataset(test_data, test_labels)

# Lead data into data loaders
batchSize = 16
train_loader = DataLoader(train_dataT, batch_size=batchSize, shuffle=True, drop_last=True, generator=torch.Generator(device))
test_loader = DataLoader(test_dataT, batch_size=test_dataT.tensors[0].shape[0], generator=torch.Generator(device))